In [88]:
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

In [89]:
rates = pd.read_excel(r'C:\Users\Moin Dalvi\Documents\Data Science Material\Data Analyst\Cointab challenge\Courier Company - Rates.xlsx')
rates = rates.T.reset_index()
rates.rename({'index':'type_of_shipment',0:'rates'},axis=1,inplace=True)
rates['zone_c'] = rates['type_of_shipment'].str.split('_').str[1]
rates['type_of_shipment'] = rates['type_of_shipment'].str.split('_').str[0] + ' '\
                           + rates['type_of_shipment'].str.split('_').str[2]
rates

,type_of_shipment,rates,zone_c
0,fwd fixed,29.5,a
1,fwd additional,23.6,a
2,fwd fixed,33.0,b
3,fwd additional,28.3,b
4,fwd fixed,40.1,c
5,fwd additional,38.9,c
6,fwd fixed,45.4,d
7,fwd additional,44.8,d
8,fwd fixed,56.6,e
9,fwd additional,55.5,e


In [90]:
order_report = pd.read_excel(r'C:\Users\Moin Dalvi\Documents\Data Science Material\Data Analyst\Cointab challenge\Company X - Order Report.xlsx')
order_report.rename({'ExternOrderNo':'order_id','SKU':'sku','Order Qty':'order_qty'}, axis=1, inplace=True)
pincode_zones = pd.read_excel(r'C:\Users\Moin Dalvi\Documents\Data Science Material\Data Analyst\Cointab challenge\Company X - Pincode Zones.xlsx')
pincode_zones.rename({'Warehouse Pincode':'warhouse_pincode','Customer Pincode':'customer_pincode','Zone':'zone_x'}, axis=1, inplace=True)
sku_master = pd.read_excel(r'C:\Users\Moin Dalvi\Documents\Data Science Material\Data Analyst\Cointab challenge\Company X - SKU Master.xlsx')
sku_master.rename({'SKU':'sku','Weight (g)':'weight_per_x_g'}, axis=1, inplace=True)
invoice = pd.read_excel(r'C:\Users\Moin Dalvi\Documents\Data Science Material\Data Analyst\Cointab challenge\Courier Company - Invoice.xlsx')
invoice.rename({'AWB Code':'awb_code','Order ID':'order_id',\
                'Charged Weight':'total_charged_weight_per_c_kg',\
                'Warehouse Pincode':'warehouse_pincode',\
                'Customer Pincode':'customer_pincode','Zone':'zone_c',\
                'Type of Shipment':'type_of_shipment',\
                'Billing Amount (Rs.)':'charged_bill_per_c'}, axis=1, inplace=True)

In [91]:
table = pd.merge(order_report, sku_master, on = 'sku', how='inner')
table.drop('sku', axis=1, inplace=True)
table

,order_id,order_qty,weight_per_x_g
0,2001827036,1.0,127
1,2001821995,1.0,127
2,2001819252,1.0,127
3,2001816996,1.0,127
4,2001814580,1.0,127
...,...,...,...
396,2001806616,1.0,250
397,2001806567,1.0,160
398,2001806567,1.0,150
399,2001806408,2.0,552


In [92]:
table.isnull().sum()

order_id          0
order_qty         0
weight_per_x_g    0
dtype: int64

In [93]:
table.isnull().sum().any()

False

In [94]:
table.order_id.value_counts()

2001809820    14
2001827036     9
2001821185     8
2001817093     8
2001812854     8
              ..
2001807012     1
2001806686     1
2001821766     1
2001806226     1
2001807328     1
Name: order_id, Length: 124, dtype: int64

In [95]:
table.order_id.nunique()

124

In [96]:
table[table.duplicated(keep='first')]

,order_id,order_qty,weight_per_x_g
72,2001827036,1.0,500
97,2001825261,4.0,112
98,2001823564,2.0,112
100,2001817093,2.0,112
101,2001812854,2.0,112
...,...,...,...
366,2001811305,1.0,210
367,2001809917,1.0,210
374,2001809820,1.0,140
382,2001808675,1.0,170


In [97]:
table = table.groupby('order_id').sum().reset_index()

In [98]:
table

,order_id,order_qty,weight_per_x_g
0,2001806210,2.0,220
1,2001806226,2.0,240
2,2001806229,3.0,500
3,2001806232,8.0,377
4,2001806233,2.0,245
...,...,...,...
119,2001821995,2.0,477
120,2001822466,10.0,352
121,2001823564,6.0,336
122,2001825261,13.0,611


In [99]:
invoice

,awb_code,order_id,total_charged_weight_per_c_kg,warehouse_pincode,customer_pincode,zone_c,type_of_shipment,charged_bill_per_c
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0


In [100]:
invoice.order_id.nunique()

124

In [101]:
tables = pd.merge(invoice, table, on = 'order_id', how = 'inner')

In [102]:
tables

,awb_code,order_id,total_charged_weight_per_c_kg,warehouse_pincode,customer_pincode,zone_c,type_of_shipment,charged_bill_per_c,order_qty,weight_per_x_g
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,8.0,377
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,4.0,615
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,12.0,929
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2.0,700
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,1.0,240
...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,3.0,500
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,3.0,607
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,2.0,515
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,4.0,689


In [103]:
tables[tables['awb_code']==1091117222124]

,awb_code,order_id,total_charged_weight_per_c_kg,warehouse_pincode,customer_pincode,zone_c,type_of_shipment,charged_bill_per_c,order_qty,weight_per_x_g
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8.0,377


In [104]:
tables[tables['awb_code']==1091117221940]

,awb_code,order_id,total_charged_weight_per_c_kg,warehouse_pincode,customer_pincode,zone_c,type_of_shipment,charged_bill_per_c,order_qty,weight_per_x_g
33,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,2.0,220


In [105]:
print(math.ceil(0.15 * 10 / 5) * 5 / 10)

0.5


In [106]:
print(math.ceil(0.73 * 10 / 5) * 5 / 10)

1.0


In [107]:
tables.isnull().sum()

awb_code                         0
order_id                         0
total_charged_weight_per_c_kg    0
warehouse_pincode                0
customer_pincode                 0
zone_c                           0
type_of_shipment                 0
charged_bill_per_c               0
order_qty                        0
weight_per_x_g                   0
dtype: int64

In [108]:
tables[tables.duplicated()]

,awb_code,order_id,total_charged_weight_per_c_kg,warehouse_pincode,customer_pincode,zone_c,type_of_shipment,charged_bill_per_c,order_qty,weight_per_x_g


In [109]:
tables['total_weight_per_x_kg'] = round(tables.weight_per_x_g / 1000, 2)
tables

,awb_code,order_id,total_charged_weight_per_c_kg,warehouse_pincode,customer_pincode,zone_c,type_of_shipment,charged_bill_per_c,order_qty,weight_per_x_g,total_weight_per_x_kg
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,8.0,377,0.38
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,4.0,615,0.62
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,12.0,929,0.93
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,2.0,700,0.70
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,1.0,240,0.24
...,...,...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,3.0,500,0.50
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,3.0,607,0.61
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,2.0,515,0.52
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,4.0,689,0.69


In [110]:
tables['weight_slab_per_c_kg'] = tables['total_charged_weight_per_c_kg'].apply(lambda x: (math.ceil(x * 10 / 5) * 5 / 10 ))

In [111]:
tables['weight_slab_per_x_kg'] = tables['total_weight_per_x_kg'].apply(lambda x: (math.ceil(x * 10 / 5) * 5 / 10 ))

In [112]:
tables = tables[['order_id','awb_code','total_weight_per_x_kg','weight_slab_per_x_kg','total_charged_weight_per_c_kg','weight_slab_per_c_kg','customer_pincode','zone_c','type_of_shipment','charged_bill_per_c']]
tables

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,total_charged_weight_per_c_kg,weight_slab_per_c_kg,customer_pincode,zone_c,type_of_shipment,charged_bill_per_c
0,2001806232,1091117222124,0.38,0.5,1.30,1.5,507101,d,Forward charges,135.0
1,2001806273,1091117222194,0.62,1.0,1.00,1.0,486886,d,Forward charges,90.2
2,2001806408,1091117222931,0.93,1.0,2.50,2.5,532484,d,Forward charges,224.6
3,2001806458,1091117223244,0.70,1.0,1.00,1.0,143001,b,Forward charges,61.3
4,2001807012,1091117229345,0.24,0.5,0.15,0.5,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...,...,...
119,2001812941,1091118551656,0.50,0.5,0.73,1.0,325207,d,Forward charges,90.2
120,2001809383,1091117614452,0.61,1.0,0.50,0.5,303702,d,Forward and RTO charges,86.7
121,2001820978,1091120922803,0.52,1.0,0.50,0.5,313301,d,Forward charges,45.4
122,2001811475,1091121844806,0.69,1.0,0.50,0.5,173212,b,Forward charges,33.0


In [113]:
tables.isnull().any()

order_id                         False
awb_code                         False
total_weight_per_x_kg            False
weight_slab_per_x_kg             False
total_charged_weight_per_c_kg    False
weight_slab_per_c_kg             False
customer_pincode                 False
zone_c                           False
type_of_shipment                 False
charged_bill_per_c               False
dtype: bool

In [114]:
tables[tables.duplicated(keep='first')]

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,total_charged_weight_per_c_kg,weight_slab_per_c_kg,customer_pincode,zone_c,type_of_shipment,charged_bill_per_c


In [115]:
table_1 = pd.merge( tables, pincode_zones, on = 'customer_pincode', how = 'inner')
table_1.drop_duplicates(keep='first', inplace=True)
table_1.reset_index(drop=True,inplace=True)
table_1 = table_1[['order_id','awb_code','total_weight_per_x_kg','weight_slab_per_x_kg','total_charged_weight_per_c_kg','weight_slab_per_c_kg','charged_bill_per_c','zone_c','zone_x','type_of_shipment']]
table_1

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment
0,2001806232,1091117222124,0.38,0.5,1.30,1.5,135.0,d,d,Forward charges
1,2001806273,1091117222194,0.62,1.0,1.00,1.0,90.2,d,d,Forward charges
2,2001806408,1091117222931,0.93,1.0,2.50,2.5,224.6,d,d,Forward charges
3,2001806458,1091117223244,0.70,1.0,1.00,1.0,61.3,b,b,Forward charges
4,2001807012,1091117229345,0.24,0.5,0.15,0.5,45.4,d,d,Forward charges
...,...,...,...,...,...,...,...,...,...,...
119,2001811039,1091117904860,0.49,0.5,0.68,1.0,90.2,d,b,Forward charges
120,2001811305,1091121846136,0.75,1.0,0.50,0.5,45.4,d,b,Forward charges
121,2001812941,1091118551656,0.50,0.5,0.73,1.0,90.2,d,b,Forward charges
122,2001809383,1091117614452,0.61,1.0,0.50,0.5,86.7,d,b,Forward and RTO charges


In [116]:
table_1[table_1['awb_code'] == 1091117221940]

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment
33,2001806210,1091117221940,0.22,0.5,2.92,3.0,174.5,b,b,Forward charges


In [117]:
table_1.isnull().any()

order_id                         False
awb_code                         False
total_weight_per_x_kg            False
weight_slab_per_x_kg             False
total_charged_weight_per_c_kg    False
weight_slab_per_c_kg             False
charged_bill_per_c               False
zone_c                           False
zone_x                           False
type_of_shipment                 False
dtype: bool

In [118]:
table_1[table_1.duplicated(keep='first')].head(10)

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment


In [119]:
table_1[table_1.duplicated(keep='last')].head(10)

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment


In [120]:
pincode_zones.customer_pincode.nunique()

108

In [121]:
tables.customer_pincode.nunique()

108

In [122]:
table_1.awb_code.nunique()

124

In [123]:
table_1.awb_code.nunique()

124

In [124]:
table_1.weight_slab_per_c_kg.unique()

array([1.5, 1. , 2.5, 0.5, 2. , 3. , 4.5])

In [125]:
for i in table_1.weight_slab_per_c_kg:
    if i / 0.5 == 1.0:
        print('fixed')
    elif i / 0.5 == 2:
        print('fixed 2 times additional')
    elif i / 0.5 == 3:
        print('fixed 3 times additional')
    elif i / 0.5 == 4:
        print('fixed 4 times additional')
    elif i / 0.5 == 5:
        print('fixed 5 times additional')
    else:
        print('fixed 6 times additional')

fixed 3 times additional
fixed 2 times additional
fixed 5 times additional
fixed 2 times additional
fixed
fixed
fixed 2 times additional
fixed 3 times additional
fixed
fixed
fixed
fixed 2 times additional
fixed 2 times additional
fixed 3 times additional
fixed 2 times additional
fixed
fixed 3 times additional
fixed 2 times additional
fixed
fixed 2 times additional
fixed 2 times additional
fixed 3 times additional
fixed 2 times additional
fixed 2 times additional
fixed 2 times additional
fixed 2 times additional
fixed 2 times additional
fixed 3 times additional
fixed 3 times additional
fixed 2 times additional
fixed 4 times additional
fixed 3 times additional
fixed 2 times additional
fixed 6 times additional
fixed 2 times additional
fixed 2 times additional
fixed 2 times additional
fixed 3 times additional
fixed 2 times additional
fixed 2 times additional
fixed 2 times additional
fixed 2 times additional
fixed 3 times additional
fixed 2 times additional
fixed 3 times additional
fixed 3 

In [126]:
table_1.type_of_shipment.unique()

array(['Forward charges', 'Forward and RTO charges'], dtype=object)

In [127]:
def additional(series):
    add = []
    for i in series:
        if i / 0.5 == 1:
            add.append(i / 0.5)
        elif i / 0.5 == 2:
            add.append(i / 0.5)
        elif i / 0.5 == 3:
            add.append(i / 0.5)
        elif i / 0.5 == 4:
            add.append(i / 0.5)
        elif i / 0.5 == 5:
            add.append(i / 0.5)
        else:
            add.append(i / 0.5)
    return add

In [128]:
table_1['additional_weight_per_x'] = additional(table_1.weight_slab_per_x_kg)
table_1 = table_1[['order_id','awb_code','total_weight_per_x_kg','weight_slab_per_x_kg','additional_weight_per_x','total_charged_weight_per_c_kg','weight_slab_per_c_kg','charged_bill_per_c','zone_c','zone_x','type_of_shipment']]
table_1

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,additional_weight_per_x,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment
0,2001806232,1091117222124,0.38,0.5,1.0,1.30,1.5,135.0,d,d,Forward charges
1,2001806273,1091117222194,0.62,1.0,2.0,1.00,1.0,90.2,d,d,Forward charges
2,2001806408,1091117222931,0.93,1.0,2.0,2.50,2.5,224.6,d,d,Forward charges
3,2001806458,1091117223244,0.70,1.0,2.0,1.00,1.0,61.3,b,b,Forward charges
4,2001807012,1091117229345,0.24,0.5,1.0,0.15,0.5,45.4,d,d,Forward charges
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811039,1091117904860,0.49,0.5,1.0,0.68,1.0,90.2,d,b,Forward charges
120,2001811305,1091121846136,0.75,1.0,2.0,0.50,0.5,45.4,d,b,Forward charges
121,2001812941,1091118551656,0.50,0.5,1.0,0.73,1.0,90.2,d,b,Forward charges
122,2001809383,1091117614452,0.61,1.0,2.0,0.50,0.5,86.7,d,b,Forward and RTO charges


In [129]:
for i in table_1.additional_weight_per_x:
    if i == 1.0:
        print('fixed')
    else:
        print('additional')

fixed
additional
additional
additional
fixed
fixed
fixed
additional
fixed
fixed
fixed
additional
additional
additional
additional
fixed
additional
additional
fixed
additional
fixed
additional
additional
fixed
fixed
additional
fixed
additional
additional
fixed
additional
additional
additional
fixed
fixed
fixed
fixed
fixed
fixed
fixed
fixed
fixed
additional
fixed
fixed
additional
fixed
additional
fixed
fixed
fixed
fixed
fixed
fixed
additional
fixed
additional
additional
additional
additional
additional
fixed
additional
fixed
additional
fixed
additional
additional
fixed
additional
additional
fixed
fixed
additional
additional
additional
fixed
fixed
fixed
fixed
fixed
fixed
additional
additional
fixed
fixed
fixed
additional
fixed
fixed
fixed
fixed
fixed
additional
fixed
additional
fixed
fixed
fixed
fixed
fixed
fixed
fixed
additional
fixed
additional
fixed
fixed
fixed
additional
fixed
fixed
fixed
fixed
fixed
fixed
fixed
fixed
additional
fixed
additional
fixed
additional
additional


In [130]:
def type_shipment(series):
    add = []
    for i in series:
        if i == 1.0:
            add.append('fixed')
        else:
            add.append('additional')
    return add

In [131]:
type_shipment(table_1.additional_weight_per_x)

['fixed',
 'additional',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'additional',
 'additional',
 'additional',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'additional',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'additional',
 'additional',
 'additional',
 'additional',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'additional',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fix

In [132]:
table_1['type_of_shipment_per_x'] = type_shipment(table_1.additional_weight_per_x)
table_1

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,additional_weight_per_x,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment,type_of_shipment_per_x
0,2001806232,1091117222124,0.38,0.5,1.0,1.30,1.5,135.0,d,d,Forward charges,fixed
1,2001806273,1091117222194,0.62,1.0,2.0,1.00,1.0,90.2,d,d,Forward charges,additional
2,2001806408,1091117222931,0.93,1.0,2.0,2.50,2.5,224.6,d,d,Forward charges,additional
3,2001806458,1091117223244,0.70,1.0,2.0,1.00,1.0,61.3,b,b,Forward charges,additional
4,2001807012,1091117229345,0.24,0.5,1.0,0.15,0.5,45.4,d,d,Forward charges,fixed
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001811039,1091117904860,0.49,0.5,1.0,0.68,1.0,90.2,d,b,Forward charges,fixed
120,2001811305,1091121846136,0.75,1.0,2.0,0.50,0.5,45.4,d,b,Forward charges,additional
121,2001812941,1091118551656,0.50,0.5,1.0,0.73,1.0,90.2,d,b,Forward charges,fixed
122,2001809383,1091117614452,0.61,1.0,2.0,0.50,0.5,86.7,d,b,Forward and RTO charges,additional


In [133]:
for i in table_1.type_of_shipment:
    if i == 'Forward charges':
        print('fwd')
    else:
        print('rto')

fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
rto
fwd
fwd
fwd
rto
rto
rto
rto
rto
rto
rto
rto
rto
rto
rto
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
rto
fwd
fwd
fwd
fwd
fwd
rto
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
fwd
rto
fwd


In [134]:
def type_of_shipment(series):
    add = []
    for i in table_1.type_of_shipment:
        if i == 'Forward charges':
            add.append('fwd')
        else:
            add.append('rto')
    return add

In [135]:
type_of_shipment(table_1.type_of_shipment) 

['fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'rto',
 'fwd',
 'fwd',
 'fwd',
 'rto',
 'rto',
 'rto',
 'rto',
 'rto',
 'rto',
 'rto',
 'rto',
 'rto',
 'rto',
 'rto',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'rto',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'rto',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'fwd',
 'rto',
 'fwd']

In [136]:
type_shipment(table_1.additional_weight_per_x)

['fixed',
 'additional',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'additional',
 'additional',
 'additional',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'additional',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'additional',
 'additional',
 'additional',
 'additional',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'additional',
 'additional',
 'additional',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fixed',
 'fix

In [137]:
table_1['type_of_shipment'] = type_of_shipment(table_1.type_of_shipment)
table_1

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,additional_weight_per_x,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment,type_of_shipment_per_x
0,2001806232,1091117222124,0.38,0.5,1.0,1.30,1.5,135.0,d,d,fwd,fixed
1,2001806273,1091117222194,0.62,1.0,2.0,1.00,1.0,90.2,d,d,fwd,additional
2,2001806408,1091117222931,0.93,1.0,2.0,2.50,2.5,224.6,d,d,fwd,additional
3,2001806458,1091117223244,0.70,1.0,2.0,1.00,1.0,61.3,b,b,fwd,additional
4,2001807012,1091117229345,0.24,0.5,1.0,0.15,0.5,45.4,d,d,fwd,fixed
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001811039,1091117904860,0.49,0.5,1.0,0.68,1.0,90.2,d,b,fwd,fixed
120,2001811305,1091121846136,0.75,1.0,2.0,0.50,0.5,45.4,d,b,fwd,additional
121,2001812941,1091118551656,0.50,0.5,1.0,0.73,1.0,90.2,d,b,fwd,fixed
122,2001809383,1091117614452,0.61,1.0,2.0,0.50,0.5,86.7,d,b,rto,additional


In [138]:
table_1['type_of_shipment_per_x'] = table_1.type_of_shipment + ' ' + table_1.type_of_shipment_per_x
table_1.drop('type_of_shipment',axis=1, inplace = True)
table_1

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,additional_weight_per_x,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment_per_x
0,2001806232,1091117222124,0.38,0.5,1.0,1.30,1.5,135.0,d,d,fwd fixed
1,2001806273,1091117222194,0.62,1.0,2.0,1.00,1.0,90.2,d,d,fwd additional
2,2001806408,1091117222931,0.93,1.0,2.0,2.50,2.5,224.6,d,d,fwd additional
3,2001806458,1091117223244,0.70,1.0,2.0,1.00,1.0,61.3,b,b,fwd additional
4,2001807012,1091117229345,0.24,0.5,1.0,0.15,0.5,45.4,d,d,fwd fixed
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811039,1091117904860,0.49,0.5,1.0,0.68,1.0,90.2,d,b,fwd fixed
120,2001811305,1091121846136,0.75,1.0,2.0,0.50,0.5,45.4,d,b,fwd additional
121,2001812941,1091118551656,0.50,0.5,1.0,0.73,1.0,90.2,d,b,fwd fixed
122,2001809383,1091117614452,0.61,1.0,2.0,0.50,0.5,86.7,d,b,rto additional


In [139]:
final_table = pd.merge(table_1, rates, left_on = ['type_of_shipment_per_x', 'zone_x'], right_on = ['type_of_shipment','zone_c'], how = 'inner')
final_table.drop_duplicates(keep='first', inplace=True)
final_table.reset_index(drop=True,inplace=True)
final_table.drop(['type_of_shipment','zone_c_y'], axis=1, inplace=True)
final_table.rename({'zone_c_x':'zone_c'}, axis=1, inplace=True)
final_table

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,additional_weight_per_x,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment_per_x,rates
0,2001806232,1091117222124,0.38,0.5,1.0,1.30,1.5,135.0,d,d,fwd fixed,45.4
1,2001807012,1091117229345,0.24,0.5,1.0,0.15,0.5,45.4,d,d,fwd fixed,45.4
2,2001806686,1091117229555,0.24,0.5,1.0,0.15,0.5,45.4,d,d,fwd fixed,45.4
3,2001807186,1091117323812,0.50,0.5,1.0,0.50,0.5,45.4,d,d,fwd fixed,45.4
4,2001807290,1091117324206,0.50,0.5,1.0,0.50,0.5,45.4,d,d,fwd fixed,45.4
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,0.48,0.5,1.0,0.80,1.0,61.3,b,e,fwd fixed,56.6
120,2001819252,1091120352712,0.27,0.5,1.0,0.30,0.5,33.0,b,e,fwd fixed,56.6
121,2001808585,1091117436652,0.50,0.5,1.0,0.72,1.0,61.3,b,e,fwd fixed,56.6
122,2001811475,1091121844806,0.69,1.0,2.0,0.50,0.5,33.0,b,e,fwd additional,55.5


In [140]:
final_table['expected_charge_per_x'] = final_table.rates * final_table.additional_weight_per_x
final_table.drop('rates', axis=1, inplace=True)
final_table

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,additional_weight_per_x,total_charged_weight_per_c_kg,weight_slab_per_c_kg,charged_bill_per_c,zone_c,zone_x,type_of_shipment_per_x,expected_charge_per_x
0,2001806232,1091117222124,0.38,0.5,1.0,1.30,1.5,135.0,d,d,fwd fixed,45.4
1,2001807012,1091117229345,0.24,0.5,1.0,0.15,0.5,45.4,d,d,fwd fixed,45.4
2,2001806686,1091117229555,0.24,0.5,1.0,0.15,0.5,45.4,d,d,fwd fixed,45.4
3,2001807186,1091117323812,0.50,0.5,1.0,0.50,0.5,45.4,d,d,fwd fixed,45.4
4,2001807290,1091117324206,0.50,0.5,1.0,0.50,0.5,45.4,d,d,fwd fixed,45.4
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,0.48,0.5,1.0,0.80,1.0,61.3,b,e,fwd fixed,56.6
120,2001819252,1091120352712,0.27,0.5,1.0,0.30,0.5,33.0,b,e,fwd fixed,56.6
121,2001808585,1091117436652,0.50,0.5,1.0,0.72,1.0,61.3,b,e,fwd fixed,56.6
122,2001811475,1091121844806,0.69,1.0,2.0,0.50,0.5,33.0,b,e,fwd additional,111.0


In [141]:
final_table['difference_between_expected_and_charged'] = final_table.charged_bill_per_c - final_table.expected_charge_per_x
calculation = final_table[['order_id','awb_code','total_weight_per_x_kg','weight_slab_per_x_kg','total_charged_weight_per_c_kg','weight_slab_per_c_kg','zone_x','zone_c','expected_charge_per_x','charged_bill_per_c','difference_between_expected_and_charged']]
calculation

,order_id,awb_code,total_weight_per_x_kg,weight_slab_per_x_kg,total_charged_weight_per_c_kg,weight_slab_per_c_kg,zone_x,zone_c,expected_charge_per_x,charged_bill_per_c,difference_between_expected_and_charged
0,2001806232,1091117222124,0.38,0.5,1.30,1.5,d,d,45.4,135.0,89.6
1,2001807012,1091117229345,0.24,0.5,0.15,0.5,d,d,45.4,45.4,0.0
2,2001806686,1091117229555,0.24,0.5,0.15,0.5,d,d,45.4,45.4,0.0
3,2001807186,1091117323812,0.50,0.5,0.50,0.5,d,d,45.4,45.4,0.0
4,2001807290,1091117324206,0.50,0.5,0.50,0.5,d,d,45.4,45.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
119,2001811604,1091118004245,0.48,0.5,0.80,1.0,e,b,56.6,61.3,4.7
120,2001819252,1091120352712,0.27,0.5,0.30,0.5,e,b,56.6,33.0,-23.6
121,2001808585,1091117436652,0.50,0.5,0.72,1.0,e,b,56.6,61.3,4.7
122,2001811475,1091121844806,0.69,1.0,0.50,0.5,e,b,111.0,33.0,-78.0


In [ ]:
calculation.to_excel('')

In [142]:
for i in calculation.difference_between_expected_and_charged:
    if i == 0:
        print('correctly charged')
    elif i >= 0:
        print('Over Charged')
    else:
        print('Under Charged')

Over Charged
correctly charged
correctly charged
correctly charged
correctly charged
correctly charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Under Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Under Charged
Under Charged
Over Charged
correctly charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over Charged
Over 

In [143]:
calculation.charged_bill_per_c.iloc[1]

45.4

In [144]:
for i in range(len(calculation)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


In [145]:
for i in range(len(calculation)):
    if calculation.difference_between_expected_and_charged.iloc[i] == 0:
        print('Correctly charged {} Rs.'.format(calculation.charged_bill_per_c.iloc[i]))
    elif calculation.difference_between_expected_and_charged.iloc[i] > 0:
        print('Over Charged {} Rs.'.format(calculation.difference_between_expected_and_charged.iloc[i]))
    else:
        print('Under Charged {} Rs.'.format(abs(calculation.difference_between_expected_and_charged.iloc[i])))

Over Charged 89.6 Rs.
Correctly charged 45.4 Rs.
Correctly charged 45.4 Rs.
Correctly charged 45.4 Rs.
Correctly charged 45.4 Rs.
Correctly charged 45.4 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 89.6 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 134.4 Rs.
Over Charged 44.800000000000004 Rs.
Over Charged 0.6000000000000085 Rs.
Over Charged 135.0 Rs.
Over Charged 0.6000000000000085 Rs.
Over Charged 0.6000000000000085 Rs.
Over Charged 45.400000000000006 Rs.
Over Charged 0.6000000000000085 Rs.
Over Charged 45.400000000000006 Rs.
Over Charged 45.400000000000006 Rs.
Over Charged 90.20000000000002 Rs.
Under Charged 44.19999999999999 Rs.
Over Charged 0.6000000000000227 Rs.
Over Charged 4.69999

In [156]:
def summary(series):
    add = []
    for i in range(len(series)):
        if calculation.difference_between_expected_and_charged.iloc[i] == 0:
            add.append(calculation.charged_bill_per_c.iloc[i])
        elif calculation.difference_between_expected_and_charged.iloc[i] > 0:
            add.append(calculation.difference_between_expected_and_charged.iloc[i])
        else:
            add.append(abs(calculation.difference_between_expected_and_charged.iloc[i]))
    return add

In [157]:
def summary_str(series):
    add = []
    for i in range(len(series)):
        if calculation.difference_between_expected_and_charged.iloc[i] == 0:
            add.append('Correctly Charged')
        elif calculation.difference_between_expected_and_charged.iloc[i] > 0:
            add.append('Over Charged')
        else:
            add.append('Under Charged')
    return add

In [158]:
summary_ = pd.DataFrame()
summary_['charge_summary'] = summary_str(calculation)
summary_['charge_summary_amount'] = summary(calculation)
summary_

,charge_summary,charge_summary_amount
0,Over Charged,89.6
1,Correctly Charged,45.4
2,Correctly Charged,45.4
3,Correctly Charged,45.4
4,Correctly Charged,45.4
...,...,...
119,Over Charged,4.7
120,Under Charged,23.6
121,Over Charged,4.7
122,Under Charged,78.0


In [160]:
summary1 = summary_.groupby('charge_summary').count().rename({'charge_summary_amount':'Count'}, axis =1).reset_index()
summary2 = summary_.groupby('charge_summary').sum().rename({'charge_summary_amount':'Amount'}, axis =1).reset_index()
summary = pd.merge( summary1, summary2, on = 'charge_summary', how = 'inner')
summary

,charge_summary,Count,Amount
0,Correctly Charged,6,260.0
1,Over Charged,112,7002.1
2,Under Charged,6,327.8


In [161]:
with pd.ExcelWriter('output.xlsx') as writer:
    summary.to_excel(writer, sheet_name='Summary')
    calculation.to_excel(writer, sheet_name='Calculation')